<a href="https://colab.research.google.com/github/OshadhaAK/ML_Competition/blob/master/best_ml_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install catboost

In [ ]:
from sklearn.model_selection import StratifiedKFold
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBClassifier
from xgboost import plot_importance
from matplotlib import pyplot
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import f1_score
import warnings
warnings.simplefilter('ignore')
import pandas.testing as tm
import seaborn as sns
sns.set()
RANDOM_SEED = 8

In [ ]:
df = pd.read_csv('/content/drive/My Drive/ml/train.csv',index_col='tripid')
Xtest = pd.read_csv('/content/drive/My Drive/ml/test.csv',index_col='tripid')
submission_df = pd.read_csv("/content/drive/My Drive/ml/sample_submission.csv",index_col='tripid')

In [ ]:
df

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare,label
tripid,,,,,,,,,,,,,
189123628,10.5,834.0,56.0,0.0000,64.0,11/1/2019 0:20,11/1/2019 0:34,6.86252,79.8993,6.90330,79.8783,270.32,correct
189125358,10.5,791.0,47.0,0.0000,134.0,11/1/2019 0:56,11/1/2019 1:09,6.88589,79.8984,6.91373,79.8923,197.85,correct
189125719,10.5,1087.0,80.0,0.0000,61.0,11/1/2019 1:08,11/1/2019 1:26,6.90839,79.8651,6.93669,79.9146,301.64,correct
189127273,10.5,598.0,271.0,15.6638,68.0,11/1/2019 2:27,11/1/2019 2:37,6.92570,79.8895,6.92748,79.8971,82.30,correct
189128020,NaN,NaN,NaN,NaN,NaN,11/1/2019 3:34,11/1/2019 3:51,6.87441,79.8615,6.84478,79.9290,358.39,correct
...,...,...,...,...,...,...,...,...,...,...,...,...,...
213803193,10.5,838.0,93.0,5.4219,451.0,1/31/2020 22:07,1/31/2020 22:21,7.29073,80.6367,7.28891,80.6557,198.26,correct
213812756,10.5,2151.0,428.0,0.0000,39.0,1/31/2020 23:07,1/31/2020 23:43,6.90569,79.8516,6.95089,79.9389,581.23,correct
213813930,10.5,263.0,9.0,0.0000,110.0,1/31/2020 23:21,1/31/2020 23:25,7.09210,79.9000,7.10135,79.9017,76.20,correct


In [ ]:
df['label'] = [0 if x == "incorrect" else 1 for x in df['label']]

In [ ]:
df = df.dropna()

In [ ]:
df.isnull().sum()

additional_fare              0
duration                     0
meter_waiting                0
meter_waiting_fare           0
meter_waiting_till_pickup    0
pickup_time                  0
drop_time                    0
pick_lat                     0
pick_lon                     0
drop_lat                     0
drop_lon                     0
fare                         0
label                        0
dtype: int64

In [ ]:
df.head()

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare,label
tripid,,,,,,,,,,,,,
189123628,10.5,834.0,56.0,0.0000,64.0,11/1/2019 0:20,11/1/2019 0:34,6.86252,79.8993,6.90330,79.8783,270.32,1
189125358,10.5,791.0,47.0,0.0000,134.0,11/1/2019 0:56,11/1/2019 1:09,6.88589,79.8984,6.91373,79.8923,197.85,1
189125719,10.5,1087.0,80.0,0.0000,61.0,11/1/2019 1:08,11/1/2019 1:26,6.90839,79.8651,6.93669,79.9146,301.64,1
189127273,10.5,598.0,271.0,15.6638,68.0,11/1/2019 2:27,11/1/2019 2:37,6.92570,79.8895,6.92748,79.8971,82.30,1
189129552,10.5,3407.0,182.0,0.0000,112.0,11/1/2019 5:38,11/1/2019 6:35,7.13402,79.8969,6.91865,79.8649,1065.02,1


In [ ]:
def minkowski_distance(x1, x2, y1, y2, p):
    return ((abs(x2 - x1) ** p) + (abs(y2 - y1)) ** p) ** (1 / p)

In [ ]:
df['euclidean_distance'] = minkowski_distance(df['pick_lon'], df['drop_lon'],df['pick_lat'], df['drop_lat'], 2)
Xtest['euclidean_distance'] = minkowski_distance(Xtest['pick_lon'], Xtest['drop_lon'],Xtest['pick_lat'], Xtest['drop_lat'], 2)


In [ ]:
df['diff_lon'] = df['pick_lon'] - df['drop_lon']
df['diff_lat'] = df['pick_lat'] - df['drop_lat']

Xtest['diff_lon'] = Xtest['pick_lon'] - Xtest['drop_lon']
Xtest['diff_lat'] = Xtest['pick_lat'] - Xtest['drop_lat']

In [ ]:
df['pickup_time'] = pd.to_datetime(df['pickup_time'], errors='coerce')
df['drop_time'] = pd.to_datetime(df['drop_time'], errors='coerce')
df['pickup_time_hour'] = df['pickup_time'].dt.hour
df['drop_time_hour'] = df['drop_time'].dt.hour
df['pickup_time_minute'] = df['pickup_time'].dt.minute
df['drop_time_minute'] = df['drop_time'].dt.minute
# df['pickup_time_dayofweek'] = df['pickup_time'].dt.dayofweek.astype('float')
# df['drop_time_dayofweek'] = df['drop_time'].dt.dayofweek.astype('float')
# df['pickup_time_day'] = df['pickup_time'].dt.day

Xtest['pickup_time'] = pd.to_datetime(Xtest['pickup_time'], errors='coerce')
Xtest['drop_time'] = pd.to_datetime(Xtest['drop_time'], errors='coerce')
Xtest['pickup_time_hour'] = Xtest['pickup_time'].dt.hour
Xtest['drop_time_hour'] = Xtest['drop_time'].dt.hour
Xtest['pickup_time_minute'] = Xtest['pickup_time'].dt.minute
Xtest['drop_time_minute'] = Xtest['drop_time'].dt.minute
# Xtest['pickup_time_dayofweek'] = Xtest['pickup_time'].dt.dayofweek.astype('float')
# Xtest['drop_time_dayofweek'] = Xtest['drop_time'].dt.dayofweek.astype('float')
# Xtest['pickup_time_day'] = Xtest['pickup_time'].dt.day

In [ ]:
def calculate_direction(d_lon, d_lat):
    result = np.zeros(len(d_lon))
    l = np.sqrt(d_lon**2 + d_lat**2)
    result[d_lon>0] = (180/np.pi)*np.arcsin(d_lat[d_lon>0]/l[d_lon>0])
    idx = (d_lon<0) & (d_lat>0)
    result[idx] = 180 - (180/np.pi)*np.arcsin(d_lat[idx]/l[idx])
    idx = (d_lon<0) & (d_lat<0)
    result[idx] = -180 - (180/np.pi)*np.arcsin(d_lat[idx]/l[idx])
    return result

In [ ]:
df['direction'] = calculate_direction(df.diff_lon, df.diff_lat)
Xtest['direction'] = calculate_direction(Xtest.diff_lon, Xtest.diff_lat)

In [ ]:
df

In [ ]:
df['riding_fare_ratio'] = (df['fare']/abs(df['duration'] +1 - df['meter_waiting']))*60
Xtest['riding_fare_ratio'] = (Xtest['fare']/abs(Xtest['duration'] +1 - Xtest['meter_waiting']))*60

In [ ]:
# df['mobile_time'] = df['duration'] - df['meter_waiting']
# Xtest['mobile_time'] = Xtest['duration'] - Xtest['meter_waiting']

In [ ]:
# df['trip_fare'] = df['fare'] - df['meter_waiting_fare'] - df['additional_fare']
# Xtest['trip_fare'] = Xtest['fare'] - Xtest['meter_waiting_fare'] - Xtest['additional_fare']

In [ ]:
# df['speed'] = df['euclidean_distance']/abs(df['duration'])
# Xtest['speed'] = Xtest['euclidean_distance']/abs(Xtest['duration'])

In [ ]:
df = df.drop(['drop_time','pickup_time','diff_lon','diff_lat'], axis = 1)
Xtest = Xtest.drop(['drop_time','pickup_time','diff_lon','diff_lat'], axis = 1)

In [ ]:
np.testing.assert_array_equal(Xtest.index.values, submission_df.index.values)

In [ ]:
df

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pick_lat,pick_lon,drop_lat,drop_lon,fare,label,euclidean_distance,pickup_time_hour,drop_time_hour,pickup_time_minute,drop_time_minute,direction,riding_fare_ratio
tripid,,,,,,,,,,,,,,,,,,
189123628,10.5,834.0,56.0,0.0000,64.0,6.86252,79.8993,6.90330,79.8783,270.32,1,0.045869,0,0,20,34,-62.753420,20.820539
189125358,10.5,791.0,47.0,0.0000,134.0,6.88589,79.8984,6.91373,79.8923,197.85,1,0.028500,0,1,56,9,-77.641274,15.934228
189125719,10.5,1087.0,80.0,0.0000,61.0,6.90839,79.8651,6.93669,79.9146,301.64,1,0.057019,1,1,8,26,-150.242655,17.954762
189127273,10.5,598.0,271.0,15.6638,68.0,6.92570,79.8895,6.92748,79.8971,82.30,1,0.007806,2,2,27,37,-166.818323,15.054878
189129552,10.5,3407.0,182.0,0.0000,112.0,7.13402,79.8969,6.91865,79.8649,1065.02,1,0.217734,5,6,38,35,81.548736,19.808184
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213803193,10.5,838.0,93.0,5.4219,451.0,7.29073,80.6367,7.28891,80.6557,198.26,1,0.019087,22,22,7,21,174.528362,15.945845
213812756,10.5,2151.0,428.0,0.0000,39.0,6.90569,79.8516,6.95089,79.9389,581.23,1,0.098307,23,23,7,43,-152.626918,20.228422
213813930,10.5,263.0,9.0,0.0000,110.0,7.09210,79.9000,7.10135,79.9017,76.20,1,0.009405,23,23,21,25,-100.413825,17.929412


In [ ]:
X = df.drop('label',1)
y = df.label

In [ ]:
X

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pick_lat,pick_lon,drop_lat,drop_lon,fare,euclidean_distance,pickup_time_hour,drop_time_hour,pickup_time_minute,drop_time_minute,direction,riding_fare_ratio
tripid,,,,,,,,,,,,,,,,,
189123628,10.5,834.0,56.0,0.0000,64.0,6.86252,79.8993,6.90330,79.8783,270.32,0.045869,0,0,20,34,-62.753420,20.820539
189125358,10.5,791.0,47.0,0.0000,134.0,6.88589,79.8984,6.91373,79.8923,197.85,0.028500,0,1,56,9,-77.641274,15.934228
189125719,10.5,1087.0,80.0,0.0000,61.0,6.90839,79.8651,6.93669,79.9146,301.64,0.057019,1,1,8,26,-150.242655,17.954762
189127273,10.5,598.0,271.0,15.6638,68.0,6.92570,79.8895,6.92748,79.8971,82.30,0.007806,2,2,27,37,-166.818323,15.054878
189129552,10.5,3407.0,182.0,0.0000,112.0,7.13402,79.8969,6.91865,79.8649,1065.02,0.217734,5,6,38,35,81.548736,19.808184
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213803193,10.5,838.0,93.0,5.4219,451.0,7.29073,80.6367,7.28891,80.6557,198.26,0.019087,22,22,7,21,174.528362,15.945845
213812756,10.5,2151.0,428.0,0.0000,39.0,6.90569,79.8516,6.95089,79.9389,581.23,0.098307,23,23,7,43,-152.626918,20.228422
213813930,10.5,263.0,9.0,0.0000,110.0,7.09210,79.9000,7.10135,79.9017,76.20,0.009405,23,23,21,25,-100.413825,17.929412


In [ ]:
print (y.value_counts())

1    15442
0     1526
Name: label, dtype: int64


In [ ]:


model = CatBoostClassifier()


model.fit(X,y)


Learning rate set to 0.034513
0:	learn: 0.6411504	total: 12.8ms	remaining: 12.8s
1:	learn: 0.5934860	total: 23.3ms	remaining: 11.6s
2:	learn: 0.5550384	total: 33.7ms	remaining: 11.2s
3:	learn: 0.5198581	total: 44.2ms	remaining: 11s
4:	learn: 0.4859864	total: 54.6ms	remaining: 10.9s
5:	learn: 0.4563262	total: 65.5ms	remaining: 10.8s
6:	learn: 0.4271298	total: 76.3ms	remaining: 10.8s
7:	learn: 0.4007369	total: 87.4ms	remaining: 10.8s
8:	learn: 0.3794321	total: 97.9ms	remaining: 10.8s
9:	learn: 0.3594847	total: 109ms	remaining: 10.8s
10:	learn: 0.3418537	total: 122ms	remaining: 11s
11:	learn: 0.3240333	total: 133ms	remaining: 11s
12:	learn: 0.3089715	total: 144ms	remaining: 11s
13:	learn: 0.2970497	total: 158ms	remaining: 11.1s
14:	learn: 0.2843791	total: 169ms	remaining: 11.1s
15:	learn: 0.2722569	total: 179ms	remaining: 11s
16:	learn: 0.2622875	total: 189ms	remaining: 10.9s
17:	learn: 0.2550571	total: 202ms	remaining: 11s
18:	learn: 0.2464345	total: 213ms	remaining: 11s
19:	learn: 0.238

In [ ]:
ynew = model.predict(Xtest)


In [ ]:
ynew

array([1, 1, 1, ..., 1, 1, 1])

In [ ]:
import collections
print (collections.Counter(ynew))


Counter({1: 8153, 0: 423})


In [ ]:
res = pd.DataFrame({'tripid': Xtest.index, 'prediction': ynew})
res.to_csv("/content/drive/My Drive/ml/test16.csv", index=False)

In [ ]:
!head '/content/drive/My Drive/ml/test16.csv'

tripid,prediction
213284604,1
213286352,1
213293973,1
213294622,1
213298687,1
213299545,1
213302332,1
213302671,1
213305594,1
